In [ ]:
# 1. Ruta de archivos
# Definimos la ruta relativa de la carpeta con los datos
ruta_carpeta_meteo = os.path.join(os.getcwd(), 'datos_meteo')
# Patrón para encontrar todos los CSV
patron_meteo = os.path.join(ruta_carpeta_meteo, '*.csv')
# Buscar archivos con libreria glob
archivos_meteo = glob.glob(patron_meteo) 

# 2. Definir tipos de datos ligeros (Ahorro masivo de RAM ya que han existido problemas para su carga)
# Int8 va de -128 a 127, Int16 hasta 32767, esto es suficiente para valores como meses, días y códigos.
tipos_datos = {
    'PROVINCIA': 'int8',
    'MUNICIPIO': 'int8',
    'ESTACION': 'int16', 
    'MAGNITUD': 'int8',
    'ANO': 'int16',
    'MES': 'int8',
    'DIA': 'int8'
}

lista_dfs = []

# 3. Bucle de carga inteligente
for archivo in archivos_meteo:
    # Leemos especificando tipos
    df_temp = pd.read_csv(archivo, sep=';', encoding='latin-1', dtype=tipos_datos)
    
    # Solo nos quedamos con NO2 (12), O3 (14)
    df_temp = df_temp[df_temp['MAGNITUD'].isin([12, 14,])]
    
    lista_dfs.append(df_temp)

# 4. Concatenar
if lista_dfs:
    df_meteo = pd.concat(lista_dfs, ignore_index=True)
    print(f"Carga completada. Filas totales: {len(df_meteo)}")
    
    # Limpieza inmediata de variables temporales
    del lista_dfs
    del df_temp
    gc.collect() # Forzamos limpieza de RAM con libreria garbage collector
else:
    print("No se encontraron archivos.")

df_meteo.head()

In [ ]:
# Columnas que no varían (Identificadores)
cols_fijas = ['ESTACION', 'MAGNITUD', 'ANO', 'MES', 'DIA']

# Generamos listas de columnas H01..H24 y V01..V24 automáticamente
cols_h = [f'H{i:02d}' for i in range(1, 25)]
cols_v = [f'V{i:02d}' for i in range(1, 25)]

# 1. Melt de Valores
df_melt_h = df_aire.melt(
    id_vars=cols_fijas, 
    value_vars=cols_h, 
    var_name='hora_texto', 
    value_name='valor'
)

# 2. Melt de Validación (V/N)
df_melt_v = df_aire.melt(
    id_vars=cols_fijas, 
    value_vars=cols_v, 
    value_name='valido'
)

# 3. Asignamos la validación
df_melt_h['valido'] = df_melt_v['valido']

# 4. Filtrado de datos válidos (Borramos lo que no sirve para liberar espacio)
# Solo nos quedamos con 'V' (Válido) y borramos filas con NaN en valor
df_final = df_melt_h[(df_melt_h['valido'] == 'V') & (df_melt_h['valor'].notna())].copy()

# 5. Limpieza de memoria
del df_aire, df_melt_h, df_melt_v
gc.collect()

print(f"Transformación lista. Filas resultantes: {len(df_final)}")

df_final.head()